In [1]:
!git clone https://github.com/nikshrimali/TSAI_END2_Phase1.git

Cloning into 'TSAI_END2_Phase1'...
remote: Enumerating objects: 670, done.
remote: Counting objects: 100% (670/670), done.
remote: Compressing objects: 100% (599/599), done.
remote: Total 670 (delta 92), reused 563 (delta 58), pack-reused 0
Receiving objects: 100% (670/670), 34.13 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [2]:
%cd /content/TSAI_END2_Phase1/
!git checkout -b Assignment7-Seq2Seq
!git pull origin Assignment7-Seq2Seq

/content/TSAI_END2_Phase1
Switched to a new branch 'Assignment7-Seq2Seq'
From https://github.com/nikshrimali/TSAI_END2_Phase1
 * branch            Assignment7-Seq2Seq -> FETCH_HEAD
Updating cbb0612..6a6f362
Fast-forward
 .gitignore                                         |      2 +
 END2_Seq2seq_Class_Code.ipynb                      |    885 +
 QuoraQ&A.ipynb                                     |    398 +
 README.md                                          |     26 +-
 data/Question_Answer_Dataset_v1.2/LICENSE-S08,S09  |    451 +
 data/Question_Answer_Dataset_v1.2/README.v1.2      |     60 +
 .../S08/data/set1/a1.htm                           |    304 +
 .../S08/data/set1/a1.txt                           |    319 +
 .../S08/data/set1/a1.txt.clean                     |    123 +
 .../S08/data/set1/a10.htm                          |    298 +
 .../S08/data/set1/a10.txt                          |    308 +
 .../S08/data/set1/a10.txt.clean                    |    106 +
 .../S08/data/set1/a10o

In [3]:
!pwd

/content/TSAI_END2_Phase1


In [4]:
from torchtext.legacy.data import BucketIterator
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time

In [5]:
!nvidia-smi

Fri Jun 18 08:06:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from dataset.quora_duplicate_dataset import QuoraDuplicateQuestionsDataset
quora_dataset = QuoraDuplicateQuestionsDataset(root_directory="data", test_split=15)

In [7]:
print(f"Number of training examples: {len(quora_dataset.train_dataset.examples)}")
print(f"Number of validation examples: {len(quora_dataset.validation_dataset.examples)}")

Number of training examples: 126874
Number of validation examples: 22389


In [8]:
vars(quora_dataset.validation_dataset.examples[10])

{'source': ['why',
  'do',
  'people',
  'ask',
  'so',
  'many',
  'googleable',
  'questions',
  'on',
  'quora',
  '?'],
 'target': ['why',
  'are',
  'there',
  'so',
  'many',
  'people',
  'using',
  'quora',
  'to',
  'answer',
  'questions',
  'that',
  'can',
  'easily',
  'be',
  'found',
  'with',
  'a',
  'simple',
  'google',
  'search',
  '?']}

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
BATCH_SIZE = 256

train_iterator, valid_iterator = BucketIterator.splits(
    (quora_dataset.train_dataset, quora_dataset.validation_dataset), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key = lambda x: len(x.source),
    sort_within_batch=False)

In [12]:
from model.encoder import Encoder
from model.decoder import Decoder
from model.seq_to_seq import Seq2Seq
INPUT_DIM = len(quora_dataset.Source.vocab)
OUTPUT_DIM = len(quora_dataset.Target.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [13]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15834, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(15739, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=15739, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 23,513,211 trainable parameters


In [15]:
optimizer = optim.Adam(model.parameters())

In [16]:
TRG_PAD_IDX = quora_dataset.Target.vocab.stoi[quora_dataset.Target.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [17]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.source
        trg = batch.target
        
        optimizer.zero_grad()
        output = model(src, trg)

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [18]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.source
            trg = batch.target

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 6m 8s
	Train Loss: 5.166 | Train PPL: 175.293
	 Val. Loss: 5.167 |  Val. PPL: 175.431
Epoch: 02 | Time: 6m 7s
	Train Loss: 4.377 | Train PPL:  79.596
	 Val. Loss: 4.907 |  Val. PPL: 135.269
Epoch: 03 | Time: 6m 6s
	Train Loss: 3.858 | Train PPL:  47.373
	 Val. Loss: 4.519 |  Val. PPL:  91.721
Epoch: 04 | Time: 6m 8s
	Train Loss: 3.445 | Train PPL:  31.343
	 Val. Loss: 4.294 |  Val. PPL:  73.271
Epoch: 05 | Time: 6m 8s
	Train Loss: 3.170 | Train PPL:  23.809
	 Val. Loss: 4.071 |  Val. PPL:  58.599
Epoch: 06 | Time: 6m 7s
	Train Loss: 2.960 | Train PPL:  19.292
	 Val. Loss: 4.072 |  Val. PPL:  58.669
Epoch: 07 | Time: 6m 5s
	Train Loss: 2.774 | Train PPL:  16.028
	 Val. Loss: 3.985 |  Val. PPL:  53.773
Epoch: 08 | Time: 6m 10s
	Train Loss: 2.635 | Train PPL:  13.943
	 Val. Loss: 3.948 |  Val. PPL:  51.810
Epoch: 09 | Time: 6m 7s
	Train Loss: 2.512 | Train PPL:  12.331
	 Val. Loss: 3.889 |  Val. PPL:  48.864
Epoch: 10 | Time: 6m 7s
	Train Loss: 2.419 | Train PPL:  11.231